In [1]:
import os
import numpy as np
import xarray as xr
import earthaccess
from datetime import datetime, timedelta
import pickle
import time
import random
from tqdm import tqdm

# Authenticate
auth = earthaccess.login(persist=True)

# Parameters
bbox = (-83.62, 41.34, -82, 42.27)      # (lon_min, lat_min, lon_max, lat_max)
res = 0.01
start_date = datetime(2024, 4, 14)
end_date = datetime(2025, 5, 23)
window_size = 7
decay = 0.8

# Ensure data directories exist
os.makedirs("../Data/", exist_ok=True)
os.makedirs("../Cache/", exist_ok=True)
os.makedirs("../Images/", exist_ok=True)

# Output grid
lat_bins = np.arange(bbox[1], bbox[3] + res, res)
lon_bins = np.arange(bbox[0], bbox[2] + res, res)
lat_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])
lon_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])
nlat, nlon = len(lat_centers), len(lon_centers)

# --- Retry Helpers ---
def safe_search(short_name, temporal, bounding_box, max_retries=500):
    retries = 0
    while True:
        try:
            results = earthaccess.search_data(
                short_name=short_name,
                temporal=temporal,
                bounding_box=bounding_box
            )
            return results
        except Exception as e:
            retries += 1
            if retries >= max_retries:
                print(f"Search failed after {max_retries} retries: {e}")
                return []
            wait = 5 + random.uniform(0, 3)
            print(f"Search error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)

def safe_download(results, directory="../Data/", max_retries=5):
    retries = 0
    while True:
        try:
            paths = earthaccess.download(results, directory)
            return paths
        except Exception as e:
            retries += 1
            if retries >= max_retries:
                print(f"Download failed after {max_retries} retries: {e}")
                return []
            wait = 5 + random.uniform(0, 3)
            print(f"Download error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)
            
# Wavelengths from a reference file (S3A or S3B, any available date)
def get_reference_wavelengths():
    satellites = ["OLCIS3A_L2_EFR_OC", "OLCIS3B_L2_EFR_OC"]
    ref_file = None
    search_dates = [("2024-06-01", "2024-06-05"), ("2024-05-01", "2024-06-01"), ("2024-04-01", "2024-05-01")]

    for sat in satellites:
        for temporal in search_dates:
            print(f"Trying to find wavelengths in {sat} from {temporal}...")

            search_ref = safe_search(
                short_name=sat,
                temporal=temporal,
                bounding_box=bbox,
            )
            if search_ref:
                ref_file = safe_download(search_ref, "../Data/")[0]
                try:
                    with xr.open_dataset(ref_file, group="sensor_band_parameters") as ds:
                        if "wavelength" in ds:
                            print("found wavelength")
                            wave_all = ds["wavelength"].data
                        elif "wavelength_3d" in ds:
                            print("found wavelength_3d")
                            wave_all = ds["wavelength_3d"].data
                        else:
                            raise KeyError("No 'wavelength' variable found in reference file.")
                    print(f"Found wavelengths in {ref_file} ({len(wave_all)} channels)")
                    return wave_all
                except Exception as e:
                    print(f"Failed to extract wavelengths from {ref_file}: {e}")
    raise RuntimeError("No reference files found to retrieve wavelengths.")

# Call it
wave_all = get_reference_wavelengths()

num_channels = len(wave_all)
print(f"Found {num_channels} channels.")

# Prepare main array
total_days = (end_date - start_date).days + 1
ndarray_all = np.full((total_days, nlat, nlon, num_channels), np.nan, dtype=np.float32)

# Process day by day
for day_idx in range(total_days):
    current_date = start_date + timedelta(days=day_idx)
    window_start = current_date - timedelta(days=window_size)
    window_end = current_date - timedelta(days=1)

    print(f"\nProcessing {current_date.date()} (composite window {window_start.date()} to {window_end.date()})")

    # Search for data from both S3A and S3B
    results_s3a = safe_search(
        short_name="OLCIS3A_L2_EFR_OC",
        temporal=(window_start.strftime("%Y-%m-%d"), window_end.strftime("%Y-%m-%d")),
        bounding_box=bbox,
    )
    
    results_s3b = safe_search(
        short_name="OLCIS3B_L2_EFR_OC",
        temporal=(window_start.strftime("%Y-%m-%d"), window_end.strftime("%Y-%m-%d")),
        bounding_box=bbox,
    )
    results = results_s3a + results_s3b

    if not results:
        print("No data found for this window. Skipping to next date.")
        continue

    # Download new files
    to_download = []
    for granule in results:
        granule_id = granule['umm']['DataGranule']['ArchiveAndDistributionInformation'][0]['Name']
        granule_basename = os.path.basename(granule_id)
        cache_path = f"../Cache/{granule_basename}.nc.npz"
        if not os.path.exists(cache_path):
            to_download.append(granule)

    if to_download:
        print(f"Need to download {len(to_download)} new files for this window.")
        paths = safe_download(to_download, "../Data/")
        if not paths:
            print("No files downloaded for this window. Skipping to next date.")
            continue
    else:
        print("All files already cached. Skipping download.")
        paths = [f"../Data/{os.path.basename(granule['granule_id'])}" for granule in results]

    # Initialize sum and weight arrays
    sum_all = np.zeros((num_channels, nlat, nlon))
    weight_all = np.zeros((num_channels, nlat, nlon))

    for path in paths:
        base = os.path.basename(path)
        date_str = base.split(".")[1][:8]
        file_date = datetime.strptime(date_str, "%Y%m%d")
        delta_days = (window_end - file_date).days
        weight = decay ** delta_days if delta_days >= 0 else 0

        cache_file = f"../Cache/{base}.npz"

        if not os.path.exists(cache_file):
            print(f"Processing {base} (not in cache)")
            try:
                nav = xr.open_dataset(path, group="navigation_data")
                lat = nav["latitude"].values
                lon = nav["longitude"].values



                rrs_ds = xr.open_dataset(path, group="geophysical_data")
                lat_idx_all, lon_idx_all, ch_idx_all, val_all = [], [], [], []
                print(f"Available variables in {base}:")
                print([v for v in rrs_ds.variables])
                print("Expected Rrs variable names (based on reference wavelengths):")
                print([f"Rrs_{int(round(w))}" for w in wave_all])

                for ch_idx, wl in tqdm(list(enumerate(wave_all)), desc=f"Channels in {base}", leave=False):
                    var_name = f"Rrs_{int(round(wl))}"
                    if var_name not in rrs_ds:
                        continue
                    band = rrs_ds[var_name].values
                    mask = (
                        np.isfinite(band) &
                        (lat >= bbox[1]) & (lat <= bbox[3]) &
                        (lon >= bbox[0]) & (lon <= bbox[2])
                    )
                    lat_valid = lat[mask]
                    lon_valid = lon[mask]
                    val_valid = band[mask]

                    lat_idx = np.searchsorted(lat_bins, lat_valid) - 1
                    lon_idx = np.searchsorted(lon_bins, lon_valid) - 1

                    lat_idx_all.extend(lat_idx)
                    lon_idx_all.extend(lon_idx)
                    ch_idx_all.extend([ch_idx] * len(val_valid))
                    val_all.extend(val_valid)

                np.savez_compressed(cache_file,
                                    lat_idx=np.array(lat_idx_all, dtype=np.int16),
                                    lon_idx=np.array(lon_idx_all, dtype=np.int16),
                                    ch_idx=np.array(ch_idx_all, dtype=np.int16),
                                    val=np.array(val_all, dtype=np.float32))
                print(f"Cached data to {cache_file}")

            except Exception as e:
                print(f"Failed to process {path}: {e}")
                continue
        else:
            print(f"Using cached data for {base}")

        # Load from cache
        data = np.load(cache_file)
        lat_idx = data['lat_idx']
        lon_idx = data['lon_idx']
        ch_idx = data['ch_idx']
        val = data['val']

        for j in range(len(val)):
            if 0 <= lat_idx[j] < nlat and 0 <= lon_idx[j] < nlon:
                sum_all[ch_idx[j], lat_idx[j], lon_idx[j]] += val[j] * weight
                weight_all[ch_idx[j], lat_idx[j], lon_idx[j]] += weight

    # Finalize average
    with np.errstate(invalid="ignore", divide="ignore"):
        avg_all = sum_all / weight_all
        avg_all[weight_all == 0] = np.nan

    ndarray_all[day_idx] = np.transpose(avg_all, (1, 2, 0))

    # Delete old files
    delete_date = (current_date - timedelta(days=window_size)).strftime('%Y%m%d')
    for fname in os.listdir("../Data/"):
        if delete_date in fname and fname.endswith(".nc"):
            try:
                os.remove(os.path.join("../Data/", fname))
                print(f"Deleted old file: {fname}")
            except Exception as e:
                print(f"Could not delete {fname}: {e}")

# Save results
np.save("../Images/composite_data.npy", ndarray_all)
print("\nSaved full 4D composite data array to '../Images/composite_data_S3.npy'.")

metadata = {"wavelengths": wave_all, "lat": lat_centers, "lon": lon_centers}
with open("../Images/composite_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)
print("Saved metadata (wavelengths, lat, lon) to '../Images/composite_metadata_S3.pkl'.")


/Users/jessecox/.pyenv/versions/3.11.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trying to find wavelengths in OLCIS3A_L2_EFR_OC from ('2024-06-01', '2024-06-05')...


QUEUEING TASKS | : 100%|████████████████████████| 8/8 [00:00<00:00, 3118.73it/s]
PROCESSING TASKS | : 100%|█████████████████████| 8/8 [00:00<00:00, 40378.38it/s]
COLLECTING RESULTS | : 100%|██████████████████| 8/8 [00:00<00:00, 107892.06it/s]


found wavelength
Found wavelengths in ../Data/S3A_OLCI_EFRNT.20240602T160846.L2.OC.nc (21 channels)
Found 21 channels.

Processing 2024-04-14 (composite window 2024-04-13 to 2024-04-13)
Need to download 6 new files for this window.


QUEUEING TASKS | : 100%|█████████████████████████| 6/6 [00:00<00:00, 452.89it/s]
PROCESSING TASKS | : 100%|████████████████████████| 6/6 [00:24<00:00,  4.04s/it]
COLLECTING RESULTS | : 100%|██████████████████| 6/6 [00:00<00:00, 125829.12it/s]


...

Cached data to ../Cache/S3A_OLCI_EFRNT.20240426T153035.L2.OC.nc.npz
Using cached data for S3A_OLCI_EFRNT.20240426T152735.L2.OC.nc
Using cached data for S3A_OLCI_EFRNT.20240426T153035.L2.OC.nc
Deleted old file: S3A_OLCI_EFRNT.20240426T152735.L2.OC.nc
Deleted old file: S3A_OLCI_EFRNT.20240426T153035.L2.OC.nc

Processing 2024-04-28 (composite window 2024-04-27 to 2024-04-27)
Need to download 2 new files for this window.


QUEUEING TASKS | : 100%|█████████████████████████| 2/2 [00:00<00:00, 445.14it/s]
PROCESSING TASKS | : 100%|████████████████████████| 2/2 [00:05<00:00,  2.83s/it]
COLLECTING RESULTS | : 100%|███████████████████| 2/2 [00:00<00:00, 20164.92it/s]


Using cached data for S3B_OLCI_EFRNT.20240427T160327.L2.OC.nc
Using cached data for S3B_OLCI_EFRNT.20240427T160327.L2.OC.nc
Deleted old file: S3B_OLCI_EFRNT.20240427T160327.L2.OC.nc

Processing 2024-04-29 (composite window 2024-04-28 to 2024-04-28)
Need to download 6 new files for this window.


QUEUEING TASKS | : 100%|█████████████████████████| 6/6 [00:00<00:00, 629.79it/s]
PROCESSING TASKS | :  17%|████                    | 1/6 [00:09<00:47,  9.41s/it]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x106d6ec50>>
Traceback (most recent call last):
  File "/Users/jessecox/.pyenv/versions/3.11.2/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [5]:

start_date = datetime(2024, 4, 14)
end_date = datetime(2024, 4, 15)

current_date = start_date
window_start = current_date - timedelta(days=1)
window_end = current_date - timedelta(days=1)

results = safe_search(
    short_name="OLCIS3A_L2_EFR_OC",
    temporal=(window_start.strftime("%Y-%m-%d"), window_end.strftime("%Y-%m-%d")),
    bounding_box=bbox,
)
paths = safe_download(results, "../Data/")
path = paths[0]
nav = xr.open_dataset(path, group="navigation_data")
lat = nav["latitude"].values
lon = nav["longitude"].values

data = xr.open_dataset(path)
keys = data.keys()
for k in keys:
    print(k)

rrs_ds = xr.open_dataset(path, group="geophysical_data")
lat_idx_all, lon_idx_all, ch_idx_all, val_all = [], [], [], []
print(f"Available variables in {base}:")
print([v for v in rrs_ds.variables])
print("Expected Rrs variable names (based on reference wavelengths):")
print([f"Rrs_{int(round(w))}" for w in wave_all])

QUEUEING TASKS | : 100%|████████████████████████| 2/2 [00:00<00:00, 2333.41it/s]
PROCESSING TASKS | : 100%|█████████████████████| 2/2 [00:00<00:00, 16288.56it/s]
COLLECTING RESULTS | : 100%|███████████████████| 2/2 [00:00<00:00, 27776.85it/s]


Available variables in S3B_OLCI_EFRNT.20240427T160327.L2.OC.nc:
['aot_865', 'angstrom', 'Rrs_400', 'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_510', 'Rrs_560', 'Rrs_620', 'Rrs_665', 'Rrs_674', 'Rrs_681', 'Rrs_709', 'chlor_a', 'Kd_490', 'l2_flags']
Expected Rrs variable names (based on reference wavelengths):
['Rrs_400', 'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_510', 'Rrs_560', 'Rrs_620', 'Rrs_665', 'Rrs_674', 'Rrs_681', 'Rrs_709', 'Rrs_754', 'Rrs_762', 'Rrs_765', 'Rrs_768', 'Rrs_779', 'Rrs_865', 'Rrs_884', 'Rrs_899', 'Rrs_939', 'Rrs_1013']


In [1]:
wave_all

NameError: name 'wave_all' is not defined

In [7]:
import h5netcdf
with h5netcdf.File(paths[0]) as file:
    groups = list(file)
groups

['sensor_band_parameters',
 'scan_line_attributes',
 'geophysical_data',
 'navigation_data',
 'processing_control']

In [25]:
for group in groups:
    ds = xr.open_dataset(path, group = group)
    print()
    print(group)
    print('---------------------------')
    for v in ds.variables:
        print(v)


sensor_band_parameters
---------------------------
wavelength
vcal_gain
vcal_offset
F0
aw
bbw
k_oz
k_no2
Tau_r

scan_line_attributes
---------------------------
year
day
msec
detnum
mside
slon
clon
elon
slat
clat
elat
csol_z

geophysical_data
---------------------------
aot_865
angstrom
Rrs_400
Rrs_412
Rrs_443
Rrs_490
Rrs_510
Rrs_560
Rrs_620
Rrs_665
Rrs_674
Rrs_681
Rrs_709
chlor_a
Kd_490
l2_flags

navigation_data
---------------------------
longitude
latitude
cntl_pt_cols
cntl_pt_rows
tilt

processing_control
---------------------------


In [35]:
ds = xr.open_dataset(path, group = 'sensor_band_parameters')
for v in ds.variables:
    print(v)
ds

wavelength
vcal_gain
vcal_offset
F0
aw
bbw
k_oz
k_no2
Tau_r


<xarray.Dataset> Size: 840B
Dimensions:      (number_of_bands: 21, number_of_reflective_bands: 21)
Dimensions without coordinates: number_of_bands, number_of_reflective_bands
Data variables:
    wavelength   (number_of_bands) float64 168B ...
    vcal_gain    (number_of_reflective_bands) float32 84B ...
    vcal_offset  (number_of_reflective_bands) float32 84B ...
    F0           (number_of_reflective_bands) float32 84B ...
    aw           (number_of_reflective_bands) float32 84B ...
    bbw          (number_of_reflective_bands) float32 84B ...
    k_oz         (number_of_reflective_bands) float32 84B ...
    k_no2        (number_of_reflective_bands) float32 84B ...
    Tau_r        (number_of_reflective_bands) float32 84B ...